In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
from functools import partial

from pisa_abw.administrative_area import AdministrativeArea
from pisa_abw.facilities import Facilities
from pisa_abw.population import WorldpopPopulation
from pisa_abw.population_served_by_isopolygons import get_population_served_by_isopolygons
from pisa_abw.visualisation import (
    plot_facilities,
    plot_isochrones,
    plot_population,
    plot_population_heatmap,
)

from optimization import jg_opt


In [ ]:
import os

from dotenv import load_dotenv

# load environment variables from an `.env` file in the local root directory
load_dotenv()

CBC_SOLVER_PATH = os.getenv("CBC_SOLVER_PATH")  # path to the cbc executable (e.g. /opt/homebrew/bin/cbc)


# Pisa Showcase, Example Notebook

### Use Case Example notebook

This notebook shows how to use the PISA package to calculate new locations for hospitals where it would have most impact, i.e. reach the highest possible number of people that currently do not have a hospital in their vicinity.

The example is based on the case of `Baucau, Timor-Leste`, and we are interested in the part of the population that cannot reach a hospital when driving 10 km. WorldPop is used as source to get the population count in `Baucau` area and the road network from OSM is used to identify their distance to current hospitals and potential locations for new facilities. Depending on the specified budget, the optimization will return a number of locations where new hospitals would reach the highest number of people that currently do not have a hospital in their vicinity.

### Define Administrative Area

We need to define the administrative area that we want to work with. This is done in two steps:
1. Create an `AdministrativeArea` object with the country name and administrative level of interest.
2. Call the `get_admin_area_boundaries` method on this `AdministrativeArea` object with the name of the region of interest to get the boundaries of the administrative area. The regions that can be specified here are dependent on which administrative level was specified in the first step.

In [ ]:
timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

### Facilities

#### Get existing facilities (in our case, hospitals) from OSM
We first identify the existing hospitals in `Baucau` by calling the `get_existing_facilities` method on the `Facilities` class, which takes the administrative area boundaries as input. This method fetches the existing facilities from OpenStreetMap (OSM).

In [ ]:
# Suppress user warning about geometry in geographic CRS. Centroid is calculated over a single facility (e.g. a hospital),
# so distances are very small and projection isn't necessary
warnings.filterwarnings(
    "ignore",
    message="Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect",
    category=UserWarning,
)

hospitals_df = Facilities(admin_area_boundaries=baucau).get_existing_facilities()

In [ ]:
plot_facilities(hospitals_df, baucau)

#### Estimate potential locations for new facilities
We then identify potential locations for new hospitals in `Baucau` by drawing a grid over the area. The `spacing` parameter specifies the distance between potential facilities. In this case, we use a spacing of 0.05.

In [ ]:
potential_hospitals_df = Facilities(admin_area_boundaries=baucau).estimate_potential_facilities(spacing=0.05)

### Get population 
We get the population data for `Baucau` from WorldPop by calling the `get_population_gdf` method on the `WorldpopPopulation` class, which takes the administrative area boundaries and the ISO3 country code as input parameters. The ISO3 country code is obtained from the `AdministrativeArea` object.

WorldPop always provides the most recent population data available, which is usually from the previous year. There is also a possibility to use Facebook population data in the PISA package, but the most recent data available is from 2019. In this example, we use WorldPop.

In [ ]:
population_gdf = WorldpopPopulation(
    admin_area_boundaries=baucau, iso3_country_code=timor_leste.get_iso3_country_code()
).get_population_gdf()

population_gdf.head()

In [ ]:
plot_population_heatmap(population_gdf, baucau)

In [ ]:
plot_population(population_gdf, baucau, random_sample_n=1000)

### Calculate isopolygons

We specify the parameters for determining access to existing and potential facilities.

For this we make choices on:
- distance type: in this notebook we use `length` (i.e. distance in meters) as the distance type, but it could also be `time` (i.e. travel time in minutes)
- distance values: these are the distances we want to use to determine access to hospitals. In this example, we use 2000, 5000 and 10000 meters (i.e. 2, 5 and 10 km) as the distance values.
- mode of transport: in this example, we use `driving` as the mode of transport, but it could also be `walking` or `cycling`.

Valid values for constants can be found in the script `pisa_abw.constants`

In [ ]:
DISTANCE_TYPE = "length"

DISTANCE_VALUES = [2000, 5000, 10000]

MODE_OF_TRANSPORT = "driving"

#### Using OSM

In order to determine the population that lives 10 km or less driving from an existing or potential facility, we need to get the road network from OSM. This can be done by calling the `get_osm_road_network` method on the `OsmRoadNetwork` class, which takes the administrative area boundaries, mode of transport, and distance type as input parameters. This will return the road network for the specific input parameters that can be used in the following step.

In [ ]:
from pisa_abw.osm_road_network import OsmRoadNetwork

road_network = OsmRoadNetwork(
    admin_area_boundaries=baucau,
    mode_of_transport=MODE_OF_TRANSPORT,
    distance_type=DISTANCE_TYPE,
).get_osm_road_network()

#### Calculate isopolygons for existing facilities

We calculate the areas (isopolygons) around the existing facilities that are reachable within the specified distances and transport mode. This is done by using the `calculate_isopolygons` method of the `OsmIsopolygonCalculator` class, which takes the existing facilities, distance type, distance values, and road network as input parameters.

In [ ]:
from pisa_abw.isopolygons import OsmIsopolygonCalculator

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    isopolygons_existing_facilities = OsmIsopolygonCalculator(
        facilities_df=hospitals_df,
        distance_type=DISTANCE_TYPE,
        distance_values=DISTANCE_VALUES,
        road_network=road_network,
    ).calculate_isopolygons()

In [ ]:
isopolygons_existing_facilities.head()

In [ ]:
plot_isochrones(isopolygons_existing_facilities, baucau)

#### Calculate isopolygons for potential facilities

We do the same for the potential facilities. This will give us the areas around the potential facilities that are reachable within the specified distances and transport mode.

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    isopolygons_potential_facilities = OsmIsopolygonCalculator(
        facilities_df=potential_hospitals_df,
        distance_type=DISTANCE_TYPE,
        distance_values=DISTANCE_VALUES,
        road_network=road_network,
    ).calculate_isopolygons()

#### Get population served by isopolygons

We can now calculate the population that lives within the isopolygons for both existing and potential facilities. This is done by using the `get_population_served_by_isopolygons` function, which takes the population data and the isopolygons as input parameters. We save this value in a dictionary with the distance type as key.

In [ ]:
population_served_current = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_existing_facilities
)

current = {DISTANCE_TYPE: population_served_current}

In [ ]:
population_served_potential = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_potential_facilities
)

potential = {DISTANCE_TYPE: population_served_potential}

## Prepare optimization data

Preparing the variables that go into the optimization:
1. We define the population count, which is the total population in the area of interest. This is used to calculate the number of people that can be served by the existing and potential facilities.
2. We define the budget for the optimization, which is the number of locations that can be built. In this example, we use a budget of 5, 20 and 50 locations.
3. We set the optimization method to use the CBC solver. The `jg_opt.OpenOptimize` function is used to create an optimization object with the specified solver path (defined in a .env file).

In [ ]:
population_count = population_gdf.population.values

In [ ]:
BUDGET = [
    5,
    20,
    50,
]  # budget for the optimization in terms of how many locations can be built

In [ ]:
cbc_optimize = partial(jg_opt.OpenOptimize, solver_path=CBC_SOLVER_PATH)

### Optimization

We run the optimization using the `jg_opt.Solve` function, which takes the population count, current population served by existing facilities, population that could be served by the potential facilities, distance type, budget, and optimization method as input parameters. The optimization will return the values (percentage of population that can be reached with the new facilities) and solutions (the best locations for new facilities) for the specified budgets.

In [ ]:
values, solutions = jg_opt.Solve(
    household=population_count,
    current=current,
    potential=potential,
    accessibility=DISTANCE_TYPE,
    budgets=BUDGET,
    optimize=cbc_optimize,
    type="ID",
)

In [ ]:
values

In [ ]:
solutions